# Construction of the Differential Elastic Scattering Cross-Section

In particle physics, the **cross-section** ($\sigma$) is a measure of the probability that a certain interaction (e.g., elastic scattering) will occur when two particles collide. And it can be thought of as an "effective size" of the target that the incoming particle sees.  
The basic relation between the number of observed events and the total cross-section is:

$$N = \sigma \cdot \mathcal{L}$$

where:
- $N$: number of events,
- $\sigma$: cross-section,
- $\mathcal{L}$: **luminosity**, describing how many collision opportunities occurred in the dataset.

It depends on the beam intensity, number of bunches, and focusing at the interaction point. When integrated over time, **integrated luminosity** $\mathcal{L}_{\text{int}}$ represents the total number of potential collisions.

The cross-section ($\sigma$) tells us **the overall probability** of an interaction, but in many analyses we need to know **how this probability depends on a kinematic variable**, such as the squared four-momentum transfer $|t|$. For elastic scattering:

$$
\frac{d\sigma_{\text{el}}}{dt} = \frac{1}{\mathcal{L}_{\text{int}}} \cdot \frac{dN}{dt}
$$ 

The **differential cross-section** is important because:
- It reveals the **angular dependence** of the scattering process,
- It allows testing of **theoretical models** (e.g., exponential behavior at low $|t|$ from diffraction),
- It provides access to the **total cross-section** via extrapolation to $t = 0$ (optical theorem).

In this part:
- $dN/dt$ is obtained from the fully corrected histogram of $|t|$, normalized by the bin width to give the event density per unit $|t|$,
- The **differential elastic cross-section** is then calculated,
- This distribution is fitted with an exponential function to determine $\left. \frac{d\sigma_{\text{el}}}{dt} \right|_{t=0}$, the value extrapolated to $t = 0$.


## Environment Setup 

In [ ]:
import ROOT
import numpy as np

ROOT.EnableImplicitMT() # Tell ROOT you want to go parallel

Due to varying data-taking conditions, the constants used in the cuts differ for each dataset and diagonal. Consequently, the correction factors are determined separately for each diagonal in every dataset. Therefore, the elastic scattering cross section is also calculated separately for each diagonal in each dataset.

One can chose a dataset and a diagonal by changing variables in the cell below.


In [ ]:
dataset_number = 3
diagonal_number = 1 # 1 - diagonal 45b56t, 0 - diagonal 45t56b

In [73]:
if not isinstance(dataset_number, int) or not (1 <= dataset_number <= 3):
    raise ValueError("Dataset number must be an integer 1 or 2 or 3")

if not isinstance(diagonal_number, int) or not (0 <= diagonal_number <= 1):
    raise ValueError("Diagonal has to be number 0 or 1")

### Loading the data from previous step
dataset = ROOT.ROOT.RDataFrame("TotemNtuple", f"./dataset{dataset_number}_diagonal{diagonal_number}_correction.root")
dataset = dataset.Define("t_abs", "abs(t)")

try:
    with ROOT.TFile(f"unfolding_correction.root") as file:
        unfolding_correction = file.Get(f"dataset{dataset_number}_diagonal{diagonal_number}")
        unfolding_correction.SetDirectory(0) # This line keeps histogram in RAM after closing the file
except AttributeError as e:
    print(f"Missing dependency in file unfolding_correction.root: dataset{dataset_number}_diagonall{diagonal_number}. Try running step4 analysis for chosen dataset AND diagonal.")
    raise e


dataset.Describe()

Dataframe from TChain TotemNtuple in file ./dataset3_diagonal0_correction.root

Property                Value
--------                -----
Columns in total           59
Columns from defines        1
Event loops run             0
Processing slots           10

Column                  Type            Origin
------                  ----            ------
A_fluct                 Double_t        Dataset
A_geom                  Double_t        Dataset
bunch_num               UInt_t          Dataset
correction              Double_t        Dataset
dataset_num             Int_t           Dataset
diagonal                Bool_t          Dataset
event_num               UInt_t          Dataset
input_status_bits       UInt_t          Dataset
run_no                  UInt_t          Dataset
t                       Double_t        Dataset
t_abs                   double          Define
th                      Double_t        Dataset
th_x                    Double_t        Dataset
th_x_L                

### Defining Custom Binning

Similarily to the previous notebooks lower and upper edges of the bins, which can be seen in the cell below, are exactly the same as those in the original article  [[1]](https://iopscience.iop.org/article/10.1209/0295-5075/101/21002#epl15159s5-2).

In [74]:
binning = np.array([0.0046, 0.0057, 0.0073, 0.009059, 0.01084, 0.01263, 0.01445, 0.01629, 0.01814, 0.02002, 0.02192, 0.02384, 
    0.02579, 0.02775, 0.02974, 0.03176, 0.0338, 0.03586, 0.03795, 0.04007, 0.04221, 0.04438, 0.04658, 0.04881, 
    0.05107, 0.05336, 0.05569, 0.05804, 0.06043, 0.06286, 0.06532, 0.06781, 0.07035, 0.07292, 0.07554, 0.07819, 
    0.08089, 0.08364, 0.08643, 0.08926, 0.09215, 0.09509, 0.09808, 0.1011, 0.1042, 0.1074, 0.1106, 0.1139, 0.1172, 
    0.1207, 0.1241, 0.1277, 0.1314, 0.1351, 0.1389, 0.1428, 0.1468, 0.1509, 0.1551, 0.1594, 0.1638, 0.1683, 0.173, 
    0.1778, 0.1827, 0.1878, 0.1931, 0.1985, 0.2041, 0.2099, 0.2159, 0.2221, 0.2286, 0.2353, 0.2424, 0.2497, 0.2573, 
    0.2654, 0.2738, 0.2827, 0.2921, 0.3021, 0.3127, 0.324, 0.3361, 0.3492, 0.3634, 0.3789])

## Distribution of the four-momentum transfer squared
 
In this analysis, our goal is to precisely measure how often elastic scattering occurs at different $|t|$ values, i.e., the differential cross-section $d\sigma/dt$. To do this, we first constructed the raw distribution of $|t|$ using the measured proton angles $\theta^*$:

$$|t| \approx (p \cdot \theta^*)^2$$
 
This gives us a histogram of events vs. $|t|$ — the foundation of our analysis. By using the special beam optics configuration in this analysis ($\beta^{*}$ = 90 m) the $|t|$ range can be explored down to $5\cdot10^{-3} \, \mathrm{GeV}^{2}$. The distribution of $|t|$ variable is presented in Fig.1 . What can be observed is that the correction factor effectively smooths the distribution by removing irregularities and makes it nearly linear on a logarithmic scale.

In [ ]:
background_correction = 0.992


dataset1_luminosity_integrated_ub = 68.0 
dataset1_luminosity_integrated_mb = dataset1_luminosity_integrated_ub  * 1e3
dataset1_full_efficiency_correction_45b_56t = 1.176 
dataset1_full_efficiency_correction_45t_56b = 1.145

dataset2_luminosity_integrated_ub = 8.2 
dataset2_luminosity_integrated_mb = dataset2_luminosity_integrated_ub  * 1e3
dataset2_full_efficiency_correction_45b_56t = 1.216
dataset2_full_efficiency_correction_45t_56b = 1.168

dataset3_luminosity_integrated_ub = 6.6
dataset3_luminosity_integrated_mb = dataset3_luminosity_integrated_ub  * 1e3
dataset3_full_efficiency_correction_45b_56t = 1.251
dataset3_full_efficiency_correction_45t_56b = 1.175

luminosities = (
    (dataset1_luminosity_integrated_mb),
    (dataset2_luminosity_integrated_mb),
    (dataset3_luminosity_integrated_mb)
    )

datasets_diagonals = (
    (dataset1_full_efficiency_correction_45b_56t, dataset1_full_efficiency_correction_45t_56b),
    (dataset2_full_efficiency_correction_45b_56t, dataset2_full_efficiency_correction_45t_56b),
    (dataset3_full_efficiency_correction_45b_56t, dataset3_full_efficiency_correction_45t_56b),
    )

integrated_luminosity_mb  = luminosities[dataset_number - 1]
full_efficiency_correction = datasets_diagonals[dataset_number-1][-diagonal_number+1]

In [76]:
t_dist_canvas = ROOT.TCanvas("t_dist_canvas", "t_dist_canvas", 1200, 600)


t_distribution_histogram = dataset.Histo1D((f"Dataset {dataset_number}, diagonal {diagonal_number} t_abs_dist after cuts", 
                                            f"Dataset {dataset_number}, diagonal {diagonal_number} |t| distribution after the cuts;|t| [GeV^2];dN [-]", 
                                            len(binning)-1, binning), "t_abs")

t_distribution_correction_histogram = dataset.Histo1D((f"Dataset {dataset_number}, diagonal {diagonal_number} t_abs_dist after cuts with correction", 
                                                       f"Dataset {dataset_number}, diagonal {diagonal_number} |t| distribution after the cuts, with acceptance correction;|t| [GeV^2];dN [-]", 
                                                       len(binning)-1, binning), "t_abs", "correction")

t_dist_canvas.Divide(2, 1)

t_dist_canvas.cd(1)
ROOT.gPad.SetLeftMargin(0.15)  
t_distribution_histogram.Draw()
t_distribution_histogram.SetStats(0)
t_distribution_histogram.GetYaxis().SetRangeUser(1, t_distribution_histogram.GetMaximum()*1.1)
ROOT.gPad.SetLogy()  

t_dist_canvas.cd(2)
ROOT.gPad.SetLeftMargin(0.15)  
t_distribution_correction_histogram.Draw()
t_distribution_correction_histogram.SetStats(0)
t_distribution_correction_histogram.GetYaxis().SetRangeUser(1, t_distribution_correction_histogram.GetMaximum()*1.1)
ROOT.gPad.SetLogy()  

t_dist_canvas.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: t_dist_canvas


**Figure 1.** On the left, the $|t|$ distribution after the cuts specified in the article [[1]](https://iopscience.iop.org/article/10.1209/0295-5075/101/21002#epl15159s5-2), and on the right, after the cuts and the application of the acceptance correction factor.

* Left plot: Raw distribution of $|t|$ after event selection cuts.At low $|t|$ (small angles), some protons fall outside the detector coverage. Therefore, the observed distribution underestimates the true number of events at small $|t|$.
* Right plot: The same distribution corrected for acceptance, showing the effect of detector geometry and beam divergence.

By dividing the distributions by the bin width, we obtain the plots shown in Fig. 2.

In [77]:
dNdt_vs_dt_histogram = dataset.Histo1D(
    (f"Dataset {dataset_number}, diagonal {diagonal_number} ", 
     f"Dataset {dataset_number}, diagonal {diagonal_number} ;|t| [GeV^2];dN/dt [1/GeV^2]", 
     len(binning)-1, binning),
    "t_abs", "correction"
)

dNdt_vs_dt_histogram.Scale(1, "width")
dNdt_vs_dt_histogram.Sumw2()

dNdt_vs_dt_histogram_unfolding = dNdt_vs_dt_histogram.Clone(
    f"Dataset {dataset_number}, diagonal {diagonal_number}   after unfolding"
)
dNdt_vs_dt_histogram_unfolding.SetTitle(
    f"Dataset {dataset_number}, diagonal {diagonal_number} after the unfolding"
)
dNdt_vs_dt_histogram_unfolding.Multiply(unfolding_correction)
dNdt_vs_dt_histogram_unfolding.Sumw2()
dNdt_vs_dt_histogram_unfolding.Scale(background_correction * full_efficiency_correction)


dNdt_vs_dt_canvas = ROOT.TCanvas("dNdt_vs_dt_canvas", "", 1200, 600)
dNdt_vs_dt_canvas.Divide(2)


dNdt_vs_dt_canvas.cd(1)
dNdt_vs_dt_histogram.Draw()
dNdt_vs_dt_histogram.SetStats(0)
ROOT.gPad.SetLogy()  

dNdt_vs_dt_canvas.cd(2)
dNdt_vs_dt_histogram_unfolding.Draw()
dNdt_vs_dt_histogram_unfolding.SetStats(0)
ROOT.gPad.SetLogy()  

dNdt_vs_dt_canvas.Draw()


Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: dNdt_vs_dt_canvas


**Figure 2.** Distributions of $|t|$ after dividing by bin width before applying the unfolding correction factors (left) and after (right).

This figure shows the transformation of the $|t|$ distribution from a partially corrected form into a fully unfolded and efficiency-corrected differential distribution.

## Elastic Scattering Cross-Section

Finally, after dividing the corrected $dN/dt$ distribution by the integrated luminosity taken from the official TOTEM results [[1]](https://iopscience.iop.org/article/10.1209/0295-5075/101/21002#epl15159s5-2), we obtain the **differential elastic scattering cross-section** ($d\sigma_{\text{el}}/dt$)  shown in Fig. 3.

This quantity is fitted with an exponential function:

$$\frac{d\sigma_{\text{el}}}{dt} = \left. \frac{d\sigma_{\text{el}}}{dt} \right|_{t=0} e^{-B|t|}$$

Here:
- $\left. \frac{d\sigma_{\text{el}}}{dt} \right|_{t=0}$ is the value of the differential cross-section **at zero momentum transfer** ($|t|=0$),
- $B$ is the **slope parameter**, describing the exponential fall-off of the cross-section with $|t|$.

The exponential dependence $e^{-B|t|}$ is motivated by the diffraction picture of elastic scattering: if the proton’s transverse matter distribution is approximately Gaussian in impact parameter space, the Fourier transform to momentum transfer space naturally yields an exponential fall-off in $|t|$. In this model, the slope parameter $B$ is directly related to the spatial size of the interaction region.

The extrapolation to $t=0$ is essential because the **optical theorem** connects the **total cross-section** $\sigma_{\text{tot}}$ to the value of $d\sigma_{\text{el}}/dt$ at $t=0$.

Since $t=0$ cannot be measured directly (detector acceptance does not extend to exactly zero scattering angle), we measure $d\sigma_{\text{el}}/dt$ at small $|t|$ and use the exponential fit to **extrapolate** to $t=0$ in a model-dependent but well-motivated way. This allows to access $\sigma_{\text{tot}}$ and compare results to other experiments and theoretical predictions.

In [80]:

dsigma_vs_dt_histogram = dNdt_vs_dt_histogram_unfolding.Clone()
dsigma_vs_dt_histogram.SetTitle(f"Dataset {dataset_number}, diagonal {diagonal_number};|t| [GeV^{2}];d#sigma_{{el}}/dt [mb/GeV^{2}]")
dsigma_vs_dt_histogram.Scale(1 / integrated_luminosity_mb)


dsigma_vs_dt_fit_canvas = ROOT.TCanvas("dsigma_vs_dt_fit_canvas", "", 800, 600)


fit_low_x_GeV2 = 0.005  # values in the article: 0.005 or 0.02
fit_high_x_GeV2 = 0.2   # values in the article: 0.1 or 0.2


fit_function = ROOT.TF1(f"Dataset {dataset_number}_fit_function", "[0]*exp([1]*x)", fit_low_x_GeV2, fit_high_x_GeV2)
dsigma_vs_dt_histogram.Fit(fit_function, "0R")# TODO explain why 0R is needed


dsigma_vs_dt_histogram.Draw()
fit_function.Draw("same")

dsigma_vs_dt_fit_canvas.SetLogy()
dsigma_vs_dt_fit_canvas.Draw()


print(f"Dataset {dataset_number} Extrapolated value of f(0) equals: {fit_function(0.0):.2f}")


Dataset 3 Extrapolated value of f(0) equals: 499.03
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      61.4462
NDf                       =           64
Edm                       =  1.12516e-05
NCalls                    =          101
p0                        =      499.029   +/-   4.08345     
p1                        =     -19.8962   +/-   0.119055    


Warning in <TCanvas::Constructor>: Deleting canvas with same name: dsigma_vs_dt_fit_canvas


**Figure 3.** The differential proton-proton elastic scattering cross section with fitted exponential function and results of the fit. 

Extrapolated value at $t = 0$:
* f(0) = 499.03 ± 4.08 $mb/GeV^2$
This is the differential elastic cross-section at zero momentum transfer.
It cannot be measured directly because the detectors do not have acceptance exactly at $t=0$, so it is obtained by extrapolating the exponential fit. This value is a key input to the optical theorem for determining the total cross-section $\sigma{\text{tot}}$.

Slope parameter:
* B = 19.896 ± 0.119 $\mathrm{GeV}^{-2}$ from $p1 = -19.896$
The slope $B$ describes how steeply the cross-section falls with $|t|$ in the low-$t$ region. Larger $B$ means a slower fall-off, corresponding to a larger effective interaction radius in the proton-proton system.

In [79]:
with ROOT.TFile(f"dsigma_dt.root", "UPDATE") as file:
    file.Delete(f"dataset{dataset_number}_diagonal{diagonal_number};1") # delete previous plot if exists
    dsigma_vs_dt_histogram.Write(f"dataset{dataset_number}_diagonal{diagonal_number}")